# GPT Action Library: Sharepoint（ドキュメントとして返す）

## はじめに

このページでは、特定のアプリケーション向けのGPT Actionを構築する開発者向けの手順とガイドを提供します。進める前に、まず以下の情報をよく理解しておいてください：
- [GPT Actionsの紹介](https://platform.openai.com/docs/actions)
- [GPT Actions Libraryの紹介](https://platform.openai.com/docs/actions/actions-library)
- [GPT Actionをゼロから構築する例](https://platform.openai.com/docs/actions/getting-started)

このソリューションは、Microsoft Graph APIの[検索機能](https://learn.microsoft.com/en-us/graph/api/resources/search-api-overview?view=graph-rest-1.0)と[ファイル取得](https://learn.microsoft.com/en-us/graph/api/driveitem-get?view=graph-rest-1.0\&tabs=http)機能を使用して、ユーザーがSharePointやOffice365でアクセス可能なファイルのコンテキストを基に、GPTアクションがユーザーの質問に答えることを可能にします。Azure Functionsを使用してGraph APIのレスポンスを処理し、人間が読みやすい形式に変換するか、ChatGPTが理解できる構造に整形します。このコードは方向性を示すものであり、要件に応じて修正する必要があります。

このソリューションは、Azure Function内でファイルを事前処理します。Azure Functionは、base64エンコードされたファイルではなく、テキストを返します。事前処理とテキストへの変換により、このソリューションは大きな非構造化ドキュメントに最適であり、最初のソリューションでサポートされているファイル数を超えて分析したい場合に適しています（ドキュメントは[こちら](https://platform.openai.com/docs/actions/sending-files)を参照）。

### 価値 + ビジネス活用事例

**価値**: ユーザーは、ChatGPTの自然言語機能を活用してSharePointのファイルに直接接続できるようになりました

**使用例**: 
- ユーザーが特定のトピックに関連するファイルを検索する必要がある場合
- ユーザーが文書の奥深くに埋もれている重要な質問への回答を必要とする場合

## アーキテクチャ / 例

![](../../../images/solution_2.gif)

このソリューションは、ログインしているユーザーに基づいて、Node.js Azure Functionを使用して以下を実行します：

1. ユーザーの初期質問に基づいて、ユーザーがアクセス権を持つ関連ファイルを検索する。

2. 見つかった各ファイルについて、一貫した読み取り可能な形式に変換し、すべてのテキストを取得する。

3. GPT 4o mini (gpt-4o-mini) を使用して、ユーザーの初期質問に基づいてファイルから関連テキストを抽出する。GPT 4o miniの価格については[こちら](https://openai.com/pricing#language-models)を参照 - 小さなトークンチャンクを扱うため、このステップのコストは僅少です。

4. そのデータをChatGPTに返す。GPTはその情報を使用してユーザーの初期質問に回答する。

以下のアーキテクチャ図からわかるように、最初の3つのステップはソリューション1と同じです。主な違いは、このソリューションがファイルをbase64文字列ではなくテキストに変換し、そのテキストをGPT 4o miniを使用して要約することです。

![](../../../images/solution_2_architecture.png)

## アプリケーション情報

### アプリケーションキーリンク

アプリケーションを始める前に、以下のリンクをご確認ください：
- アプリケーションWebサイト: https://www.microsoft.com/en-us/microsoft-365/sharepoint/collaboration
- アプリケーションAPI ドキュメント: https://learn.microsoft.com/en-us/previous-versions/office/developer/sharepoint-rest-reference/

### アプリケーションの前提条件

開始する前に、アプリケーション環境で以下の手順を実行していることを確認してください：
- Sharepoint環境へのアクセス
- Postman（およびAPIとOAuthの知識）
- platform.openai.comからのOpenAI APIキー

## ミドルウェア情報

[search concept files guide](https://learn.microsoft.com/en-us/graph/search-concept-files)に従うと、[Microsoft Graph Search API](https://learn.microsoft.com/en-us/graph/search-concept-files)は条件に合致するファイルへの参照を返しますが、ファイルの内容そのものは返しません。そのため、MSFTエンドポイントに直接アクセスするのではなく、ミドルウェアが必要になります。

手順：

1. 返されたファイルをループ処理し、[Download File endpoint](https://learn.microsoft.com/en-us/graph/api/driveitem-get-content?view=graph-rest-1.0\&tabs=http)または[Convert File endpoint](https://learn.microsoft.com/en-us/graph/api/driveitem-get-content-format?view=graph-rest-1.0\&tabs=http)を使用してファイルをダウンロードする

2. そのバイナリストリームを[pdf-parse](https://www.npmjs.com/package/pdf-parse)を使用して人間が読める形式のテキストに変換する

3. さらに最適化するために、現在Actionsに課している100,000文字制限に対応するため、関数内でgpt-4o-miniを使用して要約を行う

### 追加手順

#### Azure Functionの設定

1. [Azure Function クックブック](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_middleware_azure_function)の手順に従ってAzure Functionを設定する

#### 関数コードの追加

認証済みのAzure Functionが用意できたので、SharePoint / O365を検索するように関数を更新できます。

2. テスト関数に移動し、[このファイル](https://github.com/openai/openai-cookbook/blob/main/examples/chatgpt/sharepoint_azure_function/solution_two_preprocessing.js)のコードを貼り付けます。関数を保存してください。

> **このコードは方向性を示すためのものです** - そのまま動作するはずですが、あなたのニーズに合わせてカスタマイズするように設計されています（このドキュメントの最後にある例を参照してください）。

3. 左側の**設定**の下にある**構成**タブに移動して、以下の環境変数を設定します。Azure UIによっては、これが**環境変数**に直接表示される場合があります。

    1. `TENANT_ID`: 前のセクションからコピー

    2. `CLIENT_ID`: 前のセクションからコピー

    3. `OPENAI_API_KEY:` platform.openai.comでOpenAI APIキーを作成してください。

4. **開発ツール**の下にある**コンソール**タブに移動します

    1. コンソールで以下のパッケージをインストールします

       1. `npm install @microsoft/microsoft-graph-client`

       2. `npm install axios`

       3. `npm install pdf-parse`

       4. `npm install openai`

5. これが完了したら、Postmanから再度関数を呼び出し（POST呼び出し）、以下を本文に入力してテストしてください（レスポンスが生成されると思われるクエリと検索語を使用）。

    ```json
    {
        "query": "<質問を選択>",
        "searchTerm": "<検索語を選択>"
    }
    ```

6. レスポンスが得られたら、Custom GPTでの設定準備が完了です！

## 詳細なウォークスルー

以下では、Azure Functionでファイルの前処理と要約抽出を行うこのソリューション特有のセットアップ手順とウォークスルーについて説明します。完全なコードは[こちら](https://github.com/openai/openai-cookbook/blob/main/examples/chatgpt/sharepoint_azure_function/solution_two_preprocessing.js)で確認できます。

### コードウォークスルー

#### 認証の実装

以下では、関数内で使用するいくつかのヘルパー関数を紹介します。

#### Microsoft Graph Clientの初期化
アクセストークンを使用してGraphクライアントを初期化する関数を作成します。これはOffice 365とSharePointを検索するために使用されます。

```javascript
const { Client } = require('@microsoft/microsoft-graph-client');

function initGraphClient(accessToken) {
    return Client.init({
        authProvider: (done) => {
            done(null, accessToken);
        }
    });
}
```

#### On-Behalf-Of (OBO) トークンの取得
この関数は既存のベアラートークンを使用して、MicrosoftのIDプラットフォームからOBOトークンを要求します。これにより、認証情報を渡すことで、検索がログインユーザーがアクセス可能なファイルのみを返すことが保証されます。

```javascript
const axios = require('axios');
const qs = require('querystring');

async function getOboToken(userAccessToken) {
    const { TENANT_ID, CLIENT_ID, MICROSOFT_PROVIDER_AUTHENTICATION_SECRET } = process.env;
    const params = {
        client_id: CLIENT_ID,
        client_secret: MICROSOFT_PROVIDER_AUTHENTICATION_SECRET,
        grant_type: 'urn:ietf:params:oauth:grant-type:jwt-bearer',
        assertion: userAccessToken,
        requested_token_use: 'on_behalf_of',
        scope: 'https://graph.microsoft.com/.default'
    };

    const url = `https\://login.microsoftonline.com/${TENANT_ID}/oauth2/v2.0/token`;
    try {
        const response = await axios.post(url, qs.stringify(params), {
            headers: { 'Content-Type': 'application/x-www-form-urlencoded' }
        });
        return response.data.access\_token;
    } catch (error) {
        console.error('Error obtaining OBO token:', error.response?.data || error.message);
        throw error;
    }
}
```

#### O365 / SharePointアイテムからのコンテンツ取得

この関数はドライブアイテムのコンテンツを取得し、異なるファイルタイプを処理し、テキスト抽出のために必要に応じてファイルをPDFに変換します。PDFには[ダウンロードエンドポイント](https://learn.microsoft.com/en-us/graph/api/driveitem-get-content?view=graph-rest-1.0\&tabs=http)を使用し、その他のサポートされているファイルタイプには[変換エンドポイント](https://learn.microsoft.com/en-us/graph/api/driveitem-get-content-format?view=graph-rest-1.0\&tabs=http)を使用します。

```javascript
const getDriveItemContent = async (client, driveId, itemId, name) => {
    try {
        const fileType = path.extname(name).toLowerCase();
        // the below files types are the ones that are able to be converted to PDF to extract the text. See https://learn.microsoft.com/en-us/graph/api/driveitem-get-content-format?view=graph-rest-1.0&tabs=http
        const allowedFileTypes = ['.pdf', '.doc', '.docx', '.odp', '.ods', '.odt', '.pot', '.potm', '.potx', '.pps', '.ppsx', '.ppsxm', '.ppt', '.pptm', '.pptx', '.rtf'];
        // filePath changes based on file type, adding ?format=pdf to convert non-pdf types to pdf for text extraction, so all files in allowedFileTypes above are converted to pdf
        const filePath = `/drives/${driveId}/items/${itemId}/content` + ((fileType === '.pdf' || fileType === '.txt' || fileType === '.csv') ? '' : '?format=pdf');
        if (allowedFileTypes.includes(fileType)) {
            response = await client.api(filePath).getStream();
            // The below takes the chunks in response and combines
            let chunks = [];
            for await (let chunk of response) {
                chunks.push(chunk);
            }
            let buffer = Buffer.concat(chunks);
            // the below extracts the text from the PDF.
            const pdfContents = await pdfParse(buffer);
            return pdfContents.text;
        } else if (fileType === '.txt') {
            // If the type is txt, it does not need to create a stream and instead just grabs the content
            response = await client.api(filePath).get();
            return response;
        }  else if (fileType === '.csv') {
            response = await client.api(filePath).getStream();
            let chunks = [];
            for await (let chunk of response) {
                chunks.push(chunk);
            }
            let buffer = Buffer.concat(chunks);
            let dataString = buffer.toString('utf-8');
            return dataString
            
    } else {
        return 'Unsupported File Type';
    }
     
    } catch (error) {
        console.error('Error fetching drive content:', error);
        throw new Error(`Failed to fetch content for ${name}: ${error.message}`);
    }
};
```

#### テキスト分析のためのGPT 4o miniの統合

この関数はOpenAI SDKを利用して、ドキュメントから抽出されたテキストを分析し、ユーザークエリに基づいて関連情報を見つけます。これにより、ユーザーの質問に関連するテキストのみがGPTに返されることが保証されます。

```javascript
const getRelevantParts = async (text, query) => {
    try {
        // We use your OpenAI key to initialize the OpenAI client
        const openAIKey = process.env["OPENAI_API_KEY"];
        const openai = new OpenAI({
            apiKey: openAIKey,
        });
        const response = await openai.chat.completions.create({
            // Using gpt-4o-mini due to speed to prevent timeouts. You can tweak this prompt as needed
            model: "gpt-4o-mini",
            messages: [
                {"role": "system", "content": "You are a helpful assistant that finds relevant content in text based on a query. You only return the relevant sentences, and you return a maximum of 10 sentences"},
                {"role": "user", "content": `Based on this question: **"${query}"**, get the relevant parts from the following text:*****\n\n${text}*****. If you cannot answer the question based on the text, respond with 'No information provided'`}
            ],
            // using temperature of 0 since we want to just extract the relevant content
            temperature: 0,
            // using max_tokens of 1000, but you can customize this based on the number of documents you are searching. 
            max_tokens: 1000
        });
        return response.choices[0].message.content;
    } catch (error) {
        console.error('Error with OpenAI:', error);
        return 'Error processing text with OpenAI' + error;
    }
};
```

#### リクエストを処理するAzure Functionの作成

これらすべてのヘルパー関数を用意したので、Azure Functionはユーザーの認証、検索の実行、検索結果の反復処理を通じてテキストを抽出し、GPTに関連するテキスト部分を取得するフローを調整します。

**HTTPリクエストの処理：** 関数はHTTPリクエストからqueryとsearchTermを抽出することから始まります。Authorizationヘッダーが存在するかチェックし、ベアラートークンを抽出します。

**認証：** ベアラートークンを使用して、上記で定義したgetOboTokenを使用してMicrosoftのIDプラットフォームからOBOトークンを取得します。

**Graph Clientの初期化：** OBOトークンを使用して、上記で定義したinitGraphClientを使用してMicrosoft Graphクライアントを初期化します。

**ドキュメント検索：** 検索クエリを構築し、Microsoft Graph APIに送信してsearchTermに基づいてドキュメントを検索します。

**ドキュメント処理：** 検索で返された各ドキュメントに対して：

- getDriveItemContentを使用してドキュメントコンテンツを取得します。

- ファイルタイプがサポートされている場合、getRelevantPartsを使用してコンテンツを分析し、クエリに基づいて関連情報を抽出するためにテキストをOpenAIのモデルに送信します。

- 分析結果を収集し、ドキュメント名やURLなどのメタデータを含めます。

**レスポンス：** 関数は結果を関連性でソートし、HTTPレスポンスで返送します。

```javascript
module.exports = async function (context, req) {
    const query = req.query.query || (req.body && req.body.query);
    const searchTerm = req.query.searchTerm || (req.body && req.body.searchTerm);
    if (!req.headers.authorization) {
        context.res = {
            status: 400,
            body: 'Authorization header is missing'
        };
        return;
    }
    /// The below takes the token passed to the function, to use to get an OBO token.
    const bearerToken = req.headers.authorization.split(' ')[1];
    let accessToken;
    try {
        accessToken = await getOboToken(bearerToken);
    } catch (error) {
        context.res = {
            status: 500,
            body: `Failed to obtain OBO token: ${error.message}`
        };
        return;
    }
    // Initialize the Graph Client using the initGraphClient function defined above
    let client = initGraphClient(accessToken);
    // this is the search body to be used in the Microsft Graph Search API: https://learn.microsoft.com/en-us/graph/search-concept-files
    const requestBody = {
        requests: [
            {
                entityTypes: ['driveItem'],
                query: {
                    queryString: searchTerm
                },
                from: 0,
                // the below is set to summarize the top 10 search results from the Graph API, but can configure based on your documents. 
                size: 10
            }
        ]
    };

    try { 
        // Function to tokenize content (e.g., based on words). 
        const tokenizeContent = (content) => {
            return content.split(/\s+/);
        };

        // Function to break tokens into 10k token windows for gpt-4o-mini
        const breakIntoTokenWindows = (tokens) => {
            const tokenWindows = []
            const maxWindowTokens = 10000; // 10k tokens
            let startIndex = 0;

            while (startIndex < tokens.length) {
                const window = tokens.slice(startIndex, startIndex + maxWindowTokens);
                tokenWindows.push(window);
                startIndex += maxWindowTokens;
            }

            return tokenWindows;
        };
        // This is where we are doing the search
        const list = await client.api('/search/query').post(requestBody);

        const processList = async () => {
            // This will go through and for each search response, grab the contents of the file and summarize with gpt-4o-mini
            const results = [];

            await Promise.all(list.value[0].hitsContainers.map(async (container) => {
                for (const hit of container.hits) {
                    if (hit.resource["@odata.type"] === "#microsoft.graph.driveItem") {
                        const { name, id } = hit.resource;
                        // We use the below to grab the URL of the file to include in the response
                        const webUrl = hit.resource.webUrl.replace(/\s/g, "%20");
                        // The Microsoft Graph API ranks the reponses, so we use this to order it
                        const rank = hit.rank;
                        // The below is where the file lives
                        const driveId = hit.resource.parentReference.driveId;
                        const contents = await getDriveItemContent(client, driveId, id, name);
                        if (contents !== 'Unsupported File Type') {
                            // Tokenize content using function defined previously
                            const tokens = tokenizeContent(contents);

                            // Break tokens into 10k token windows
                            const tokenWindows = breakIntoTokenWindows(tokens);

                            // Process each token window and combine results
                            const relevantPartsPromises = tokenWindows.map(window => getRelevantParts(window.join(' '), query));
                            const relevantParts = await Promise.all(relevantPartsPromises);
                            const combinedResults = relevantParts.join('\n'); // Combine results

                            results.push({ name, webUrl, rank, contents: combinedResults });
                        } 
                        else {
                            results.push({ name, webUrl, rank, contents: 'Unsupported File Type' });
                        }
                    }
                }
            }));

            return results;
        };
        let results;
        if (list.value[0].hitsContainers[0].total == 0) {
            // Return no results found to the API if the Microsoft Graph API returns no results
            results = 'No results found';
        } else {
            // If the Microsoft Graph API does return results, then run processList to iterate through.
            results = await processList();
            results.sort((a, b) => a.rank - b.rank);
        }
        context.res = {
            status: 200,
            body: results
        };
    } catch (error) {
        context.res = {
            status: 500,
            body: `Error performing search or processing results: ${error.message}`,
        };
    }
};
```

### カスタマイズ

以下はカスタマイズ可能な潜在的な領域です。

- 何も見つからない場合に一定回数再検索するようにGPTプロンプトをカスタマイズできます。

- 検索クエリをカスタマイズして、特定のSharePointサイトやO365ドライブのみを検索するようにコードをカスタマイズできます。これにより検索を集中させ、検索精度を向上させることができます。現在の設定では、ログインユーザーがアクセス可能なすべてのファイルを検索します。

- gpt-4o-miniの代わりにgpt-4oを使用できます。これによりコストとレイテンシがわずかに増加しますが、より高品質な要約が得られる可能性があります。

- Microsoft Graphの呼び出し内で検索するファイル数をカスタマイズできます。

### 考慮事項

100K文字以下の返却と[45秒のタイムアウト](https://platform.openai.com/docs/actions/production/timeouts)に関して、Actionsのすべての同じ制限がここでも適用されることに注意してください。

- これはテキストのみに機能し、画像には対応していません。Azure Function内で追加のコードを使用することで、GPT-4oを使用して画像の要約を抽出するようにカスタマイズできます。

- これは構造化データには対応していません。構造化データがユースケースの主要部分である場合は、ソリューション1を推奨します。

## ChatGPT ステップ

### カスタムGPT指示

Custom GPTを作成したら、以下のテキストをInstructionsパネルにコピーしてください。質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

In [ ]:
You are a Q&A helper that helps answer users questions. You have access to a documents repository through your API action. When a user asks a question, you pass in that question exactly as stated to the "query" parameter, and for the "searchTerm" you use a single keyword or term you think you should use for the search.

****

Scenario 1: There are answers

If your action returns results, then you take the results from the action and summarize concisely with the webUrl returned from the action. You answer the users question to the best of your knowledge from the action

****

Scenario 2: No results found

If the response you get from the action is "No results found", stop there and let the user know there were no results and that you are going to try a different search term, and explain why. You must always let the user know before conducting another search.

Example:

****

I found no results for "DEI". I am now going to try [insert term] because [insert explanation]

****

Then, try a different searchTerm that is similar to the one you tried before, with a single word. 

Try this three times. After the third time, then let the user know you did not find any relevant documents to answer the question, and to check SharePoint. Be sure to be explicit about what you are searching for at each step.

****

In either scenario, try to answer the user's question. If you cannot answer the user's question based on the knowledge you find, let the user know and ask them to go check the HR Docs in SharePoint. If the file is a CSV, XLSX, or XLS, you can tell the user to download the file using the link and re-upload to use Advanced Data Analysis.

### OpenAPI スキーマ

Custom GPTを作成したら、以下のテキストをActionsパネルにコピーしてください。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

以下の仕様では、前処理に情報を提供するために`query`パラメータを渡し、Microsoft Graphで適切なファイルを見つけるために`searchTerm`を渡します。
>上記のスクリーンショットでコピーしたリンクに基づいて、Function Appの名前、関数名、およびコードを必ず変更してください

In [ ]:
openapi: 3.1.0
info:
  title: SharePoint Search API
  description: API for searching SharePoint documents.
  version: 1.0.0
servers:
  - url: https://{your_function_app_name}.azurewebsites.net/api
    description: SharePoint Search API server
paths:
  /{your_function_name}?code={enter your specific endpoint id here}:
    post:
      operationId: searchSharePoint
      summary: Searches SharePoint for documents matching a query and term.
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              properties:
                query:
                  type: string
                  description: The full query to search for in SharePoint documents.
                searchTerm:
                  type: string
                  description: A specific term to search for within the documents.
      responses:
        '200':
          description: Search results
          content:
            application/json:
              schema:
                type: array
                items:
                  type: object
                  properties:
                    documentName:
                      type: string
                      description: The name of the document.
                    snippet:
                      type: string
                      description: A snippet from the document containing the search term.
                    url:
                      type: string
                      description: The URL to access the document.

## 認証手順

以下は、このサードパーティアプリケーションとの認証設定に関する手順です。ご質問がありますか？この手順の詳細については、[Getting Started Example](https://platform.openai.com/docs/actions/getting-started)をご確認ください。

*認証に関するより詳細な手順については、上記および[Azure Function cookbook](https://cookbook.openai.com/examples/chatgpt/gpt_actions_library/gpt_middleware_azure_function)を参照してください。*

## FAQ & トラブルシューティング

- なぜコード内で[SharePoint API](https://learn.microsoft.com/en-us/sharepoint/dev/sp-add-ins/get-to-know-the-sharepoint-rest-service?tabs=csom)ではなくMicrosoft Graph APIを使用しているのですか？

  - SharePoint APIはレガシーです - Microsoftのドキュメント[こちら](https://learn.microsoft.com/en-us/sharepoint/dev/apis/sharepoint-rest-graph)によると、「SharePoint Onlineでは、SharePointに対するREST APIを使用したイノベーションは、Microsoft Graph REST APIを通じて推進されています。」Graph APIはより柔軟性を提供し、SharePoint APIでも[Microsoft Graph APIと直接やり取りする代わりに、なぜこれが必要なのですか？](#why-is-this-necessary-instead-of-interacting-with-the-microsoft-api-directly)セクションに記載されているのと同じファイルの問題が発生します。

- これはどのような種類のファイルをサポートしていますか？
    1. これは、Convert Fileエンドポイントのドキュメント[_こちら_](https://learn.microsoft.com/en-us/graph/api/driveitem-get-content-format?view=graph-rest-1.0\&tabs=http)に記載されているすべてのファイルをサポートしています。具体的には、_pdf, doc, docx, odp, ods, odt, pot, potm, potx, pps, ppsx, ppsxm, ppt, pptm, pptx, rtf_をサポートしています。

    2. 検索結果がXLS、XLSX、またはCSVを返す場合、これによりユーザーはファイルをダウンロードし、Advanced Data Analysisを使用して質問するために再アップロードするよう促されます。上記で述べたように、構造化データがユースケースの一部である場合は、ソリューション1を推奨します。

- なぜOBOトークンをリクエストする必要があるのですか？

  - Azure Functionへの認証に使用するのと同じトークンをGraph APIへの認証に使用しようとすると、「無効なオーディエンス」トークンエラーが発生します。これは、トークンのオーディエンスがuser\_impersonationのみに設定できるためです。

  - これに対処するため、この関数は[On Behalf Ofフロー](https://learn.microsoft.com/en-us/entra/identity-platform/v2-oauth2-on-behalf-of-flow)を使用して、アプリ内でFiles.Read.Allにスコープされた新しいトークンをリクエストします。これにより、ログインしたユーザーの権限が継承され、この関数はログインしたユーザーがアクセス権を持つファイルのみを検索することになります。

  - Azure Function Appsはステートレスであることを意図しているため、各リクエストで意図的に新しいOn Behalf Ofトークンをリクエストしています。シークレットを保存してプログラム的に取得するために、これをAzure Key Vaultと統合することも可能です。

*優先的に対応してほしい統合機能はありますか？統合機能にエラーがありますか？GitHubでPRやissueを作成していただければ、確認いたします。*